### chatcompletition, Prompt Template, LLM api call with Open Source LLMs,with local storage

In [3]:
import minsearch
import json
from dotenv import load_dotenv
from groq import Groq
import os

load_dotenv()

True

In [4]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

documents=[]

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)
        
Index = minsearch.Index(
    text_fields = ['question','section','text'],
    keyword_fields = ['course']
)  

Index.fit(documents)

In [5]:
def search(query):
    boost = {'question': 3, 'section' : 0.4}

    results = Index.search(
        query=query,
        filter_dict={'course':'mlops-zoomcamp'},
        boost_dict=boost,
        num_results=5
)
    return results

In [6]:
def build_prompt(query, search_results):
    prompt_template = """ 
You are an expert machine learning and mlops engineering helping a junior engineer as an assitant and guide. 
Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering. DO NOT USE OTHER CONTENT OTHER THAN GIVEN CONTEXT!
if the CONTEXT does not contain the answer, Output "Not FOUND in the context given" and explain your answer with reasons.

QUESTION: {question}

CONTEXT: {context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [7]:
def llm_call(prompt):
    client = Groq(
    api_key=os.getenv("GROQ_API_KEY"),
)
    response = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": prompt,
        }
    ],
    model="llama3-8b-8192",
)
    return print(response.choices[0].message.content)

In [8]:
Query = "How to use mlflow for experiment tracking?"
def rag(query):
    
    search_results = search(query)
    Prompt = build_prompt(query, search_results)
    answer = llm_call(Prompt)
    return answer

### Mistral-7B openSource LLM

In [9]:
from transformers import AutoModelForCausalLM
from transformers import AutoTokenizer

In [10]:
from huggingface_hub import login

In [11]:
login(os.getenv("HF_zoomcamp"))

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /teamspace/studios/this_studio/.cache/huggingface/token
Login successful


In [12]:
model = AutoModelForCausalLM.from_pretrained(
    "mistralai/Mistral-7B-v0.1", device_map="auto", load_in_4bit=True
)

tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-v0.1", padding_side="left")


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [13]:
model_inputs = tokenizer(["A list of colors: red, blue"], return_tensors="pt").to("cuda")
generated_ids = model.generate(**model_inputs)
tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/transformers/generation/utils.py:1259: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/bitsandbytes/nn/modules.py:435: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(


'A list of colors: red, blue, green, yellow, orange, purple, pink,'

In [20]:
def llm_call(prompt):
    model_inputs = tokenizer([prompt], return_tensors="pt").to("cuda")
    generated_ids = model.generate(**model_inputs,max_length=1500)
    result = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    return result

In [21]:
rag("How do i start using mlflow?")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


'You are an expert machine learning and mlops engineering helping a junior engineer as an assitant and guide. \nAnswer the QUESTION based on the CONTEXT from the FAQ database.\nUse only the facts from the CONTEXT when answering. DO NOT USE OTHER CONTENT OTHER THAN GIVEN CONTEXT!\nif the CONTEXT does not contain the answer, Output "Not FOUND in the context given" and explain your answer with reasons.\n\nQUESTION: How do i start using mlflow?\n\nCONTEXT: section: General course questions\nquestion: Course - How do I start?\nanswer: No matter if you\'re with a \'live\' cohort or follow on the self-paced mode, the way to start is similar:\nsee what things are where by reading  pins and  bookmarks on the course-channel\nreading the repo (bookmarked in channel) and watching the video lessons (playlist bookmarked in channel)\nif have questions, search the channel itself first; someone may have already asked and gotten a solution\nif don\'t want to comb through the search results, read  for th

In [ ]:
print(_)